In this notebook we will discuss what you need to know and do for project 2 :)

**Important:** formulas are better viewed if you start `jupyter notebook` (as opposed to use github's visualization).

In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import libitg
from libitg import Symbol, Terminal, Nonterminal, Span
from libitg import Rule, CFG
from libitg import FSA
from collections import defaultdict

# Lexicon

I am going to use a very simple lexicon as running example, you would instead use the translation pairs we provided, but note that

* we provided you with [all translation pairs under IBM1 in both directions](https://uva-slpl.github.io/nlp2/resources/project_crf/lexicon.tgz), thus there are many pairs in there
* you should retain only the top scoring translation pairs (this will make your lexicon smaller and your forests more manageable)
* we suggest 5 translations per source word
* you need to explicitly encode insertion and deletion in your lexicon, you can bootstrap that from alignments to NULL


In [60]:
lexicon = defaultdict(set)
lexicon['le'].update(['the', '-EPS-'])  # we will assume that `le` can be deleted
lexicon['-EPS-'].update(['a', 'the'])  # we will assume that `the` and `a` can be inserted
lexicon['e'].add('and')
lexicon['chien'].add('dog')
lexicon['noir'].update(['black', 'noir'])  
lexicon['blanc'].add('white')
lexicon['petit'].update(['small', 'little'])
lexicon['petite'].update(['small', 'little'])

# Play with the parser

You should play with the basic data structures and algorithms we provided you with, namely

* Symbol: Terminal, Nonterminal, Span
* Rule
* CFG
* FSA, make_fsa, LengthConstraint
* ITG-related functions: make_source_itg, make_target_side_itg
* [Earley intersection](https://uva-slpl.github.io/nlp2/resources/papers/Aziz-Earley.pdf): earley

you need to know what they do and when to use them to obtain something you need.


# ITG

We deal with ITGs by

* constructing a source-language CFG
* parsing a source string
* projecting the resulting forest onto the target-language vocabulary through the rules of an ITG and the lexicon


**Note on performance:** we could in principle instantiate a different source-side CFG for each source sentence by constraining the lexicon to source words that occur in the sentence we are translating.
That is to say that if our sentence does not contain a word (e.g. *petite*) there is no point in including that word in the CFG.

In [61]:
# this function uses the entire lexicon
src_cfg = libitg.make_source_side_itg(lexicon)

In [62]:
print(src_cfg)

[X] ||| [X] [X]
[X] ||| 'petit'
[X] ||| 'e'
[X] ||| 'le'
[X] ||| 'chien'
[X] ||| 'blanc'
[X] ||| 'noir'
[X] ||| 'petite'
[X] ||| '-EPS-'
[S] ||| [X]


# FSA

We represent a sentence as a linear-chain FSA.

In [63]:
src_fsa = libitg.make_fsa('le chien noir')
print(src_fsa)

states=4
initial=0
final=3
arcs=3
origin=0 destination=1 label=le
origin=1 destination=2 label=chien
origin=2 destination=3 label=noir


# Forests

Forests represent sets of derivations, they are represented by CFGs where symbols have been decorated with spans.
We are working with Earley parser, which takes a CFG, an FSA, the CFG's starting symbol, and the symbol that should be used as the starting symbol of the resulting CFG.
Note that Earley takes a CFG and returns another!

The most basic forest we need is one that contains all derivations of the source.

In [64]:
src_forest = libitg.earley(src_cfg, src_fsa, 
                           start_symbol=Nonterminal('S'), 
                           sprime_symbol=Nonterminal("D(x)"))
print(src_forest)

[X]:1-3 ||| [X]:1-1 [X]:1-3
[X]:1-3 ||| [X]:1-3 [X]:3-3
[X]:1-3 ||| [X]:1-2 [X]:2-3
[X]:0-2 ||| [X]:0-0 [X]:0-2
[X]:0-2 ||| [X]:0-1 [X]:1-2
[X]:0-2 ||| [X]:0-2 [X]:2-2
[X]:3-3 ||| '-EPS-':3-3
[X]:3-3 ||| [X]:3-3 [X]:3-3
[X]:0-3 ||| [X]:0-1 [X]:1-3
[X]:0-3 ||| [X]:0-2 [X]:2-3
[X]:0-3 ||| [X]:0-0 [X]:0-3
[X]:0-3 ||| [X]:0-3 [X]:3-3
[X]:2-3 ||| [X]:2-2 [X]:2-3
[X]:2-3 ||| 'noir':2-3
[X]:2-3 ||| [X]:2-3 [X]:3-3
[D(x)] ||| [S]:0-3
[X]:1-1 ||| '-EPS-':1-1
[X]:1-1 ||| [X]:1-1 [X]:1-1
[S]:0-3 ||| [X]:0-3
[X]:0-0 ||| '-EPS-':0-0
[X]:0-0 ||| [X]:0-0 [X]:0-0
[X]:1-2 ||| [X]:1-1 [X]:1-2
[X]:1-2 ||| [X]:1-2 [X]:2-2
[X]:1-2 ||| 'chien':1-2
[X]:0-1 ||| [X]:0-1 [X]:1-1
[X]:0-1 ||| 'le':0-1
[X]:0-1 ||| [X]:0-0 [X]:0-1
[X]:2-2 ||| '-EPS-':2-2
[X]:2-2 ||| [X]:2-2 [X]:2-2


Then we need to project this forest to the target vocabulary by using the lexicon and the ITG template rules.
This is precisely the set \\(\mathcal D(x)\\) of our lecture notes.

In [65]:
Dx = libitg.make_target_side_itg(src_forest, lexicon)
print(Dx)

[X]:0-0 ||| 'a':0-0
[X]:0-0 ||| 'the':0-0
[X]:0-0 ||| [X]:0-0 [X]:0-0
[X]:0-2 ||| [X]:0-0 [X]:0-2
[X]:0-2 ||| [X]:0-2 [X]:0-0
[X]:0-2 ||| [X]:0-1 [X]:1-2
[X]:0-2 ||| [X]:1-2 [X]:0-1
[X]:0-2 ||| [X]:0-2 [X]:2-2
[X]:0-2 ||| [X]:2-2 [X]:0-2
[X]:3-3 ||| 'a':3-3
[X]:3-3 ||| 'the':3-3
[X]:3-3 ||| [X]:3-3 [X]:3-3
[X]:0-3 ||| [X]:0-1 [X]:1-3
[X]:0-3 ||| [X]:1-3 [X]:0-1
[X]:0-3 ||| [X]:0-2 [X]:2-3
[X]:0-3 ||| [X]:2-3 [X]:0-2
[X]:0-3 ||| [X]:0-0 [X]:0-3
[X]:0-3 ||| [X]:0-3 [X]:0-0
[X]:0-3 ||| [X]:0-3 [X]:3-3
[X]:0-3 ||| [X]:3-3 [X]:0-3
[X]:2-3 ||| [X]:2-2 [X]:2-3
[X]:2-3 ||| [X]:2-3 [X]:2-2
[X]:2-3 ||| 'black':2-3
[X]:2-3 ||| 'noir':2-3
[X]:2-3 ||| [X]:2-3 [X]:3-3
[X]:2-3 ||| [X]:3-3 [X]:2-3
[D(x)] ||| [S]:0-3
[X]:1-1 ||| 'a':1-1
[X]:1-1 ||| 'the':1-1
[X]:1-1 ||| [X]:1-1 [X]:1-1
[S]:0-3 ||| [X]:0-3
[X]:1-3 ||| [X]:1-1 [X]:1-3
[X]:1-3 ||| [X]:1-3 [X]:1-1
[X]:1-3 ||| [X]:1-3 [X]:3-3
[X]:1-3 ||| [X]:3-3 [X]:1-3
[X]:1-3 ||| [X]:1-2 [X]:2-3
[X]:1-3 ||| [X]:2-3 [X]:1-2
[X]:1-2 ||| [X]:1-1 [X]:1-2
[X]:

# Dealing with a translation observation

In training, we can observe translations, thus we need to constrain \\(\mathcal D(x)\\) to the set of derivations that in addition to \\(x\\) also produce our target observation \\(y\\).

We do that by intersecting the forest for \\(\mathcal D(x)\\) with an automaton that represents \\(y\\).

That is, we first make an FSA for the target sentence:

In [66]:
tgt_fsa = libitg.make_fsa('the black dog')
print(tgt_fsa)

states=4
initial=0
final=3
arcs=3
origin=0 destination=1 label=the
origin=1 destination=2 label=black
origin=2 destination=3 label=dog


and then parse this FSA with the projected forest we got earlier from Earley.
The resulting forest will represent the set \\(\mathcal D(x,y)\\).

In [67]:
Dxy = libitg.earley(Dx, tgt_fsa,
                    start_symbol=Nonterminal("D(x)"), 
                    sprime_symbol=Nonterminal('D(x,y)'))
print(Dxy)

[X]:0-3:1-3 ||| [X]:2-3:1-2 [X]:0-2:2-3
[X]:0-3:1-3 ||| [X]:1-3:1-3 [X]:0-1:3-3
[X]:0-3:1-3 ||| [X]:0-1:1-1 [X]:1-3:1-3
[X]:2-3:1-2 ||| 'black':2-3:1-2
[X]:2-3:0-2 ||| [X]:2-2:0-1 [X]:2-3:1-2
[X]:2-3:0-2 ||| [X]:3-3:0-1 [X]:2-3:1-2
[X]:0-1:2-2 ||| '-EPS-':0-1:2-2
[X]:0-1:1-1 ||| '-EPS-':0-1:1-1
[X]:2-2:0-1 ||| 'the':2-2:0-1
[D(x)]:0-3 ||| [S]:0-3:0-3
[X]:3-3:0-1 ||| 'the':3-3:0-1
[X]:1-3:0-3 ||| [X]:3-3:0-1 [X]:1-3:1-3
[X]:1-3:0-3 ||| [X]:1-1:0-1 [X]:1-3:1-3
[X]:1-3:0-3 ||| [X]:2-3:0-2 [X]:1-2:2-3
[X]:1-2:2-3 ||| 'dog':1-2:2-3
[X]:1-1:0-1 ||| 'the':1-1:0-1
[X]:0-1:3-3 ||| '-EPS-':0-1:3-3
[S]:0-3:0-3 ||| [X]:0-3:0-3
[D(x,y)] ||| [D(x)]:0-3
[X]:0-1:0-0 ||| '-EPS-':0-1:0-0
[X]:0-3:0-3 ||| [X]:2-3:0-2 [X]:0-2:2-3
[X]:0-3:0-3 ||| [X]:0-1:0-1 [X]:1-3:1-3
[X]:0-3:0-3 ||| [X]:0-0:0-1 [X]:0-3:1-3
[X]:0-3:0-3 ||| [X]:1-3:0-3 [X]:0-1:3-3
[X]:0-3:0-3 ||| [X]:3-3:0-1 [X]:0-3:1-3
[X]:0-3:0-3 ||| [X]:0-1:0-0 [X]:1-3:0-3
[X]:0-2:2-3 ||| [X]:1-2:2-3 [X]:0-1:3-3
[X]:0-2:2-3 ||| [X]:0-1:2-2 [X]:1-2:2-3
[

In [68]:
# this is just to illustrate that ref_forest accepts a single target (English) string
libitg.language_of_fsa(libitg.forest_to_fsa(Dxy, Nonterminal('D(x,y)')))

{'the black dog'}

# Length constraint

In order to learn our CRF for maximum likelihood we need to compute the gradient of the log-likelihood.

The gradient invoves two expectations (see lecture notes and project description). One expectation summarises all ways in which we can derive the joint observation \\((x, y)\\), the other summarises all ways in which we can derive the incomplete observation \\(x, n\\). The former is computed out of the forest that represents \\(\mathcal D(x, y)\\) and the latter is computed out of the forest that represents \\(\mathcal D_n(x)\\).

Here we will show you how to get \\(\mathcal D_n(x)\\) by parsing a special automaton that accepts the language \\(\Delta^n\\) where \\(\Delta\\) is the vocabulary of the target language.

In [69]:
# `strict` controls whether the constraint is |yield(d)| == n (strict=True) or |yield(d)| <= n (strict=False)
length_fsa = libitg.LengthConstraint(4, strict=False)

This special automaton accepts strings containing 1 to 4 words, no matter which words. The label -WILDCARD- is a special label such that `x == -WILDCARD-` evaluates to `True` no matter the value of `x`.

In [70]:
print(length_fsa)

states=5
initial=0
final=1 2 3 4
arcs=4
origin=0 destination=1 label=-WILDCARD-
origin=1 destination=2 label=-WILDCARD-
origin=2 destination=3 label=-WILDCARD-
origin=3 destination=4 label=-WILDCARD-


In [71]:
Dnx = libitg.earley(Dx, length_fsa,
                    start_symbol=Nonterminal("D(x)"), 
                    sprime_symbol=Nonterminal("D_n(x)"))
print(Dnx)

[X]:1-2:1-3 ||| [X]:1-2:1-2 [X]:2-2:2-3
[X]:1-2:1-3 ||| [X]:2-2:1-2 [X]:1-2:2-3
[X]:1-2:1-3 ||| [X]:1-2:1-2 [X]:1-1:2-3
[X]:1-2:1-3 ||| [X]:1-1:1-2 [X]:1-2:2-3
[X]:1-2:0-2 ||| [X]:1-1:0-1 [X]:1-2:1-2
[X]:1-2:0-2 ||| [X]:2-2:0-1 [X]:1-2:1-2
[X]:1-2:0-2 ||| [X]:1-2:0-1 [X]:1-1:1-2
[X]:1-2:0-2 ||| [X]:1-2:0-1 [X]:2-2:1-2
[X]:1-1:1-2 ||| 'a':1-1:1-2
[X]:1-1:1-2 ||| 'the':1-1:1-2
[X]:3-3:2-4 ||| [X]:3-3:2-3 [X]:3-3:3-4
[X]:0-1:2-4 ||| [X]:1-1:2-4 [X]:0-1:4-4
[X]:0-1:2-4 ||| [X]:0-1:2-3 [X]:0-0:3-4
[X]:0-1:2-4 ||| [X]:0-0:2-3 [X]:0-1:3-4
[X]:0-1:2-4 ||| [X]:0-1:2-3 [X]:1-1:3-4
[X]:0-1:2-4 ||| [X]:1-1:2-3 [X]:0-1:3-4
[X]:0-1:2-4 ||| [X]:0-0:2-4 [X]:0-1:4-4
[X]:0-1:2-4 ||| [X]:0-1:2-2 [X]:0-0:2-4
[X]:0-1:2-4 ||| [X]:0-1:2-2 [X]:1-1:2-4
[X]:3-3:1-2 ||| 'a':3-3:1-2
[X]:3-3:1-2 ||| 'the':3-3:1-2
[X]:1-2:0-3 ||| [X]:1-1:0-2 [X]:1-2:2-3
[X]:1-2:0-3 ||| [X]:2-2:0-2 [X]:1-2:2-3
[X]:1-2:0-3 ||| [X]:1-2:0-2 [X]:1-1:2-3
[X]:1-2:0-3 ||| [X]:1-2:0-2 [X]:2-2:2-3
[X]:1-2:0-3 ||| [X]:1-2:0-1 [X]:2-2:1-3
[X]:

In [72]:
# this produces a super large FSA which enumerates the strings in the forest
Dnx_as_fsa = libitg.forest_to_fsa(Dnx, Nonterminal('D_n(x)'))

In [73]:
# here we get the strings in a normal python set
candidates = libitg.language_of_fsa(Dnx_as_fsa)
for candidate in sorted(candidates):
    print(candidate)

a a
a a black
a a black dog
a a dog
a a dog black
a a dog noir
a a noir
a a noir dog
a black
a black a
a black a dog
a black dog
a black dog a
a black dog the
a black the
a black the dog
a dog
a dog a
a dog a black
a dog a noir
a dog black
a dog black a
a dog black the
a dog noir
a dog noir a
a dog noir the
a dog the
a dog the black
a dog the noir
a noir
a noir a
a noir a dog
a noir dog
a noir dog a
a noir dog the
a noir the
a noir the dog
a the
a the black
a the black dog
a the dog
a the dog black
a the dog noir
a the noir
a the noir dog
black a
black a a
black a a dog
black a dog
black a dog a
black a dog the
black a the
black a the dog
black dog
black dog a
black dog a a
black dog a the
black dog the
black dog the a
black dog the the
black the
black the a
black the a dog
black the dog
black the dog a
black the dog the
black the the
black the the dog
dog a
dog a a
dog a a black
dog a a noir
dog a black
dog a black a
dog a black the
dog a noir
dog a noir a
dog a noir the
dog a the
dog

In [74]:
# Let's check how many string we got
print('Strings in D_n(x): %d' % len(candidates))
# and check whether the gold-standard reference is still in the set of candidates (it should be!)
print('Does D_n(x) contain the reference (it should!)? %s' % ('yes' if 'the black dog' in candidates else 'no'))

Strings in D_n(x): 176
Does D_n(x) contain the reference (it should!)? yes


# Model

Earley parsing helped us instatiated the sets which will be support to our probability distribution, in particular,

* \\(\mathcal D_n(x)\\) will be support to our joint distribution \\(P(Y, D|X=x, N=n)\\)
* \\(\mathcal D(x, y)\\) will be support to our posterior distribution \\(P(D|X=x, Y=y, N=n)\\)

and our joint distribution corresponds to 

\begin{align}
P(Y=y, D=d|X=x, N=n) &= \frac{\exp(w^\top \Phi(y, d, x, n))}{\sum_{y' \in \mathcal Y_n(x)} \sum_{d' \in \mathcal D(x, y)} \exp(w^\top \Phi(y', d', x, n))} \\
 &= \frac{\exp(w^\top \Phi(y, d, x, n))}{\sum_{d' \in \mathcal D_n(x)} \exp(w^\top \Phi(y', d', x, n))}
\end{align}

where

* \\(\Phi\\) is a feature function that maps any tuple \\(y, d, x, n\\) to \\(\mathbb R^d\\)
* \\(\theta\\) is a weight vector in \\(\mathbb R^d\\)
* \\(\mathcal Y_n(x)\\) is the set of translations of \\(x\\) that are at most \\(n\\)-words long
* and the last equality is due to the fact that there is a deterministic mapping between a derivation and the string it projects on either language, that is, \\(y' = \text{yield}_\Delta(d')\\).

In order to instantiate the joint distribution for the entire space \\(\mathcal D_n(x)\\) we must use features that can be locally assigned to the steps of a derivation. This corresponds to the idea of having *local features* or *edge potentials*:

\begin{align}
P(Y=y, D=d|X=x, N=n) 
 &= \frac{\exp \left( \sum_{r_{s,t} \in d} w^\top \phi(r, s, t, x, n) \right)}{\sum_{r_{s, t} \in d' \in \mathcal D_n(x)} \exp\left( \sum_c w^\top \phi(r, s, t, x, n) \right)} \\
\end{align}

where
* a derivation is seen as a sequence of rules decorated with spans
* \\(r_{s,t}\\) is a rule decorated with a source and a target span (in the code sometimes we call this an *edge*)
* \\(\phi\\) is a (local) feature function that maps any tuple \\((r, s, t, x, n)\\) to \\(\mathbb R^d\\)
* note that \\( \Phi(y, d, x, n) = \sum_{r_{s, t} \in d} \phi(r, s, t, x, n) \\)

The formulation in terms of local potentials make it clear that in designing feature representations for derivations we have unrestricted access to \\(x\\), \\(n\\), the rule identiy \\(r\\), the source span \\(s\\) and the target span \\(t\\), but note that because we do not have unrestricted access to \\(y\\), the information that \\(t\\) provides is rather limited.

* For example, knowing the rule `X_{1-3,1-3} -> X_{2-3,1-2} X_{1-2,2-3}` tells us that the LHS spans from 1 to 3 on the source. Suppose, we are translating \\(x\\) = `le chien noir`, then we know the the LHS projects onto `chien noir`, that it is prefixed by `BOS le` and followed by `EOS` (end-of-sentence). On the other hand, if we focus on the target span, we only know that we will produce up 2 words (because `3-1=2`) but we dont't know which words those are;
* Another rule, `X_{2-3,1-2} -> noir/black` tells us a different story, it tells us that the lexical entry `noir` has been aligned to the lexical entry `black`, it also tell us about the positions where these words are, so we could for example compute a distortion feature similar to IBM2's jump value;
* A rule, `X_{0-1,0-0} -> le/-EPS-` tells us that a deletion has happened;
* Insertions can for example appear like this `X_{1-2,1-3} -> X_{1-1,1-2} X_{1-2,2-3}`, note that the first RHS symbol has an empty source span `1-1`, but the target span is longer than 0, this means an insertion has happened;
* The rule `X_{1-3,1-3} -> X_{2-3,1-2} X_{1-2,2-3}` also tells us that we are making an inversion, note that the source spans are not adjacent, but rather flipped, e.g. 2-3 for the first RHS symbol and 1-2 for the second RHS symbol, we could have a feature capture that;

After undestanding how to use the parser to get the relevant sets, you should focus on implement a feature function that can convert the information available on an edge into a sparse feature vector.

We suggest you start with a very simple feature function that is mostly *dense*. This will make development a lot easier in the beginning. Here we show a minimal version of it.



We start with some auxiliary code

In [83]:
def get_source_word(fsa: FSA, origin: int, destination: int) -> str:
    """Returns the python string representing a source word from origin to destination (assuming there's a single one)"""
    labels = list(fsa.labels(origin, destination))
    assert len(labels) == 1, 'Use this function only when you know the path is unambiguous, found %d labels %s for (%d, %d)' % (len(labels), labels, origin, destination)
    return labels[0]

def get_target_word(symbol: Symbol):
    """Returns the python string underlying a certain terminal (thus unwrapping all span annotations)"""
    if not symbol.is_terminal():
        raise ValueError('I need a terminal, got %s of type %s' % (symbol, type(symbol)))
    return symbol.root().obj()

def get_bispans(symbol: Span):
    """
    Returns the bispans associated with a symbol. 
    
    The first span returned corresponds to paths in the source FSA (typically a span in the source sentence),
     the second span returned corresponds to either
        a) paths in the target FSA (typically a span in the target sentence)
        or b) paths in the length FSA
    depending on the forest where this symbol comes from.
    """
    if not isinstance(symbol, Span):
        raise ValueError('I need a span, got %s of type %s' % (symbol, type(symbol)))
    s, start2, end2 = symbol.obj()  # this unwraps the target or length annotation
    _, start1, end1 = s.obj()  # this unwraps the source annotation
    return (start1, end1), (start2, end2)

and define our prototype feature function:

In [84]:
def simple_features(edge: Rule, src_fsa: FSA, eps=Terminal('-EPS-'), 
                    sparse_del=False, sparse_ins=False, sparse_trans=False) -> dict:
    """
    Featurises an edge given
        * rule and spans
        * src sentence as an FSA
        * TODO: target sentence length n
        * TODO: extract IBM1 dense features
    crucially, note that the target sentence y is not available!    
    """
    fmap = defaultdict(float)
    if len(edge.rhs) == 2:  # binary rule
        fmap['type:binary'] += 1.0
        # here we could have sparse features of the source string as a function of spans being concatenated
        (ls1, ls2), (lt1, lt2) = get_bispans(edge.rhs[0])  # left of RHS
        (rs1, rs2), (rt1, rt2) = get_bispans(edge.rhs[1])  # right of RHS        
        # TODO: double check these, assign features, add some more
        if ls1 == ls2:  # deletion of source left child
            pass
        if rs1 == rs2:  # deletion of source right child
            pass
        if ls2 == rs1:  # monotone
            pass
        if ls1 == rs2:  # inverted
            pass        
    else:  # unary
        symbol = edge.rhs[0]
        if symbol.is_terminal():  # terminal rule
            fmap['type:terminal'] += 1.0
            # we could have IBM1 log probs for the traslation pair or ins/del
            (s1, s2), (t1, t2) = get_bispans(symbol)            
            if symbol.root() == eps:  # symbol.root() gives us a Terminal free of annotation
                # for sure there is a source word
                src_word = get_source_word(src_fsa, s1, s2)                
                fmap['type:deletion'] += 1.0
                # dense versions (for initial development phase)
                # TODO: use IBM1 prob
                #ff['ibm1:del:logprob'] += 
                # sparse version
                if sparse_del:
                    fmap['del:%s' % src_word] += 1.0
            else:                  
                # for sure there's a target word
                tgt_word = get_target_word(symbol)
                if s1 == s2:  # has not consumed any source word, must be an eps rule                    
                    fmap['type:insertion'] += 1.0
                    # dense version
                    # TODO: use IBM1 prob
                    #ff['ibm1:ins:logprob'] += 
                    # sparse version
                    if sparse_ins:
                        fmap['ins:%s' % tgt_word] += 1.0
                else:
                    # for sure there's a source word
                    src_word = get_source_word(src_fsa, s1, s2)
                    fmap['type:translation'] += 1.0
                    # dense version
                    # TODO: use IBM1 prob
                    #ff['ibm1:x2y:logprob'] += 
                    #ff['ibm1:y2x:logprob'] += 
                    # sparse version                    
                    if sparse_trans:
                        fmap['trans:%s/%s' % (src_word, tgt_word)] += 1.0
        else:  # S -> X
            fmap['top'] += 1.0
    return fmap

Now we can simply featurize edges, one at a time.

In [85]:
def featurize_edges(forest, src_fsa, 
                    sparse_del=False, sparse_ins=False, sparse_trans=False,
                    eps=Terminal('-EPS-')) -> dict:
    edge2fmap = dict()
    for edge in forest:
        edge2fmap[edge] = simple_features(edge, src_fsa, eps, sparse_del, sparse_ins, sparse_trans)
    return edge2fmap

Let's have a look at what we get if we use some sparse features:

In [86]:
for edge, fmap in featurize_edges(Dxy, src_fsa, True, True, True).items():
    print(edge)
    print(fmap)
    print()

[X]:0-1:1-1 ||| '-EPS-':0-1:1-1
defaultdict(<class 'float'>, {'type:deletion': 1.0, 'del:le': 1.0, 'type:terminal': 1.0})

[X]:0-1:0-1 ||| [X]:0-1:0-0 [X]:1-1:0-1
defaultdict(<class 'float'>, {'type:binary': 1.0})

[X]:0-1:2-2 ||| '-EPS-':0-1:2-2
defaultdict(<class 'float'>, {'type:deletion': 1.0, 'del:le': 1.0, 'type:terminal': 1.0})

[X]:0-2:2-3 ||| [X]:1-2:2-3 [X]:0-1:3-3
defaultdict(<class 'float'>, {'type:binary': 1.0})

[X]:0-3:1-3 ||| [X]:0-1:1-1 [X]:1-3:1-3
defaultdict(<class 'float'>, {'type:binary': 1.0})

[X]:1-3:0-3 ||| [X]:3-3:0-1 [X]:1-3:1-3
defaultdict(<class 'float'>, {'type:binary': 1.0})

[X]:0-3:0-3 ||| [X]:0-1:0-0 [X]:1-3:0-3
defaultdict(<class 'float'>, {'type:binary': 1.0})

[X]:0-3:0-3 ||| [X]:3-3:0-1 [X]:0-3:1-3
defaultdict(<class 'float'>, {'type:binary': 1.0})

[X]:1-1:0-1 ||| 'the':1-1:0-1
defaultdict(<class 'float'>, {'type:insertion': 1.0, 'ins:the': 1.0, 'type:terminal': 1.0})

[X]:2-2:0-1 ||| 'the':2-2:0-1
defaultdict(<class 'float'>, {'type:insertion': 1

Now, recall the definition of the joint distribution

\begin{align}
P(y,d|x, n) &= \frac{\exp \left( \sum_{r_{s,t} \in d} w^\top \phi(r, s, t, x, n) \right)}{Z_n(x)}
\end{align}

for convenience, let's work with log probabilities, 

\begin{align}
\log P(y,d|x,n) &= \log \exp \left( \sum_{r_{s,t} \in d} w^\top \phi(r, s, t, x, n) \right) - \log Z_n(x) \\
  &= \underbrace{\sum_{r_{s,t} \in d} \underbrace{w^\top \phi(r, s, t, x, n)}_{\text{local log-potential}}}_{\text{log of unnormalised distribution}} - \log Z_n(x)
\end{align}

Now, in the log-domain, we can compute the logarithm of unnormalised probabilities by summing along the edges in a derivation.

This is great news! We can express the log of the unnormalised distribution by decorating edges with local log-potentials, that is, 

\begin{equation}
\omega(r_{s,t}) = w^\top \phi(r, s, t, x, n)
\end{equation}

where \\(\omega\\) takes an edge in the hypergraph and returns its local log-potential (dot product of feature weights and feature vector).


You are ready to define a weight function now:

In [87]:
def weight_function(edge, fmap, wmap) -> float:
    pass  # dot product of fmap and wmap  (working in log-domain)


# Tips

Working with hypergraphs can be tedious, they can be quite large, a few tips:

* do not parse very long sentences: discard training instances where either string is longer than say 10 words in development phase, for the final report you can try to stretch all the way to 30 or so;
* parse once: with a fixed lexicon and grammar, the parse forests will never change, thus produce them one at a time and pickle them to disk, so that whenever you need the forest during SGD it will be pre-computed
* featurise once: same applies to feature vectors, if you do not change your feature function, feature vectors will remain the same, thus, extract them once and pickle them to disk



# MLE

Now it's time you optimise your likelihood

\begin{align}
\mathcal L(w|x, y) &= \log P(y|x, n) \\
 &= \log \sum_{d \in \mathcal D(x, y)} P(y, d|x, n) \\
 &= \log \sum_{d \in \mathcal D(x, y)} \frac{\exp(w^\top \Phi(y, d|x, n))}{Z_n(x)} \\
 &= \log \frac{\sum_{d \in \mathcal D(x, y)} \exp(w^\top \Phi(y, d|x, n))}{Z_n(x)} \\
 &= \log \frac{Z(x, y)}{Z_n(x)} \\
 &= \log Z(x, y) - \log Z_n(x)
\end{align}

First of all, we need to efficiently compute
* \\(\log Z(x,y)\\) the sum of all unnormalised probabilities for derivations that explain the translation pair \\(x, y\\)
* and \\(\log Z_n(x)\\) the sum of all unnormalised probabilities for derivations that explain the incomplete observation \\(x, n\\)
now note that we have acyclic hypergraphs whose edges represent log-potentials for a model which is log-linear. This means that we can efficiently compute the global log-normalisers with a recursive formula that visits each node in the forest once. This recursion is called the `Inside algorithm`.

The *inside weight* of a node corresponds to the sum of the weights of all paths under this node in the forest.

\begin{equation}
    I(v) = 
    \begin{cases}
        1 \quad \text{if } v \text{ is terminal }\\
        \bigoplus_{e \in BS(v)} \omega(e) \bigotimes_{u \in tail(e)} I(u) \quad \text{ otherwise}
    \end{cases}
\end{equation}

where
* \\(u\\) and \\(v\\) are nodes (these are annotated symbols in the forest)
* \\(e\\) is an edge (this is an annotated rule in the forest)
* \\(\text{tail}(e)\\) is a sequence of children nodes (this is the RHS of an annotated rule in the forest)
* \\(BS(v)\\) corresponds to the set of edges that are incoming to v (this is the set of annotated rules for which v is LHS in the forest)
* \\(\otimes\\) corresponds to product of probabilities or equivalently sum of log-probabilities
* \\(\otimes\\) corresponds to sum of probabilities or equivalently log-of-sum-of-exponentiated-log-probabilities, i.e. \\(\log (\exp(a) + \exp(b))\\) where \\(a\\) and \\(b\\) are log-probabilities
* if \\(\otimes\\) and \\(\oplus\\) confused you, then you need to learn a little more about [semirings](http://www.aclweb.org/anthology/J/J99/J99-4004.pdf)

The inside recursion can be implemented iteratively by visiting the nodes in [top-sorted order](https://en.wikipedia.org/wiki/Topological_sorting).
              
Check the lecture notes and [slides](https://uva-slpl.github.io/nlp2/resources/slides/crf.pdf) for a pseudo-code of the Inside algorithm.

The inside at the root of the forest represents the log-normaliser of the forest.

Now you should be ready to implement topsort and the inside algorithm:

In [88]:
def top_sort(forest: CFG) -> list:
    """Returns ordered list of nodes according to topsort order in an acyclic forest"""
    pass

def inside_algorithm(forest: CFG, tsort: list, edge_weights: dict) -> dict:
    """Returns the inside weight of each node"""
    pass

Now, we will approach the optimisation of the log-likelihood via SGD, that is, 
we will do so by taking steps towards the steepest ascent at each training instance (or mini-batch).

Taking derivatives with respect to \\(w\\) we get

\begin{align}
\nabla_w \mathcal L(w|x, y, n) &= \mathbb E_{P(D|Y=y, X=x, N=n)}[\Phi(Y, D, X, N)] - \mathbb E_{P(Y, D|X=x, N=n)}[\Phi(Y, D, X, N)]
\end{align}

where
* the first expectation uses the forest \\(\mathcal D(x, y)\\)
* the second expectation uses the forest \\(\mathcal D_n(x)\\)

again because our CRF is edge-factored, we can compute this rather efficiently combining inside weights with *outside* weights, for which we need the *outside algorithm*. If you heard of forward-backward, then inside-outside is just a generalisation for hypergraphs (instead of graphs). You can learn more about expectations from forest from [Li and Eisner](http://www.aclweb.org/anthology/D09-1005).

Whereas the inside was defined easily on its own, the outside will be defined differently so that
\begin{align}
    I(u) \otimes O(u)
\end{align}
is proportional to the marginal probability of node \\(u\\), where the missing proportionality constant corresponds to the log-normaliser of the distribution (which is precisely the Inside weight of the root of the forest).

\begin{equation}
    O(v) = 
    \begin{cases}
        1 \quad \text{if } FS(v) =\emptyset \\
        \bigoplus_{e \in FS(v)} O(\text{head}(e)) \bigotimes_{s \in tail(e)\setminus\{v\}} I(s) \quad \text{ otherwise}
    \end{cases}
\end{equation}

where
* \\(s\\) and \\(v\\) are nodes (these are annotated symbols in the forest)
* \\(e\\) is an edge (this is an annotated rule in the forest)
* \\(\text{head}(e)\\) is a the edge's head node (this is the annotated LHS of an annotated rule in the forest)
* \\(FS(v)\\) corresponds to the set of edges that are outgoing from v (this is the set of annotated rules for which v appears in the RHS)
* \\(I(s)\\) is the inside weight of node \\(s\\)
* \\(\otimes\\) corresponds to product of probabilities or equivalently sum of log-probabilities
* \\(\otimes\\) corresponds to sum of probabilities or equivalently log-of-sum-of-exponentiated-log-probabilities, i.e. \\(\log (\exp(a) + \exp(b))\\) where \\(a\\) and \\(b\\) are log-probabilities
* if \\(\otimes\\) and \\(\oplus\\) confused you, then you need to learn a little more about [semirings](http://www.aclweb.org/anthology/J/J99/J99-4004.pdf)


Again, check the [slides](https://uva-slpl.github.io/nlp2/resources/slides/crf.pdf) for a pseu-code for the outside algorithm.

Now you should be ready to implement the *outside algorithm*.

In [89]:
def outside_algorithm(forest: CFG, tsort:list, edge_weights: dict, inside: dict) -> dict:
    """Returns the outside weight of each node"""
    pass

It's now time to combine inside and outside and efficiently compute the expected feature vector under each distribution.

This takes a linear pass over the edges of the forest, and [Li and Eisner](http://www.aclweb.org/anthology/D09-1005) taught us how to do that very efficiently (see figure 4, or our [slides](https://uva-slpl.github.io/nlp2/resources/slides/crf.pdf) for a clean pseudo-code).

You should be ready to implement it.

In [90]:
def expected_feature_vector(forest: CFG, inside: dict, outside: dict, edge_features: dict) -> dict:
    """Returns an expected feature vector (here a sparse python dictionary)"""
    pass

# SGD

Now it's time to take gradient steps,

\begin{align}
    w = w + \delta \nabla_w \mathcal L(w|x, y, n)
\end{align}

where for a certain observation \\(x, n, y\\) we compute the expected feature vectors, subtract them, scale by a learning rate and get a parameter update.
If we have a mini-batch, we accumulate the gradients.

In the beginning of the project you will be working with the simple (mostly dense) feature function, thus this should work reasonably well. At some point you will model sparse features and then you will most likely need a [regulariser](http://www.aclweb.org/anthology/P08-1024).

To diagonose convergence we typically track log-likelihood of training data, with online learning or mini-batching is typically too expensive to go over the entire trainin set computing log-likelihood, thus you can do that on each batch individually and track both the batch log-likelihood and a running average.

Lastly, you might want to experiment with averaged parameters where in parallel to the parameters \\(w\\) that you actually use, you can keep a running average of parameters \\(w_{\text{avg}}\\), at then end of learning, this running average typically shows better performance at prediction time.

# Prediction

Finally, at prediction time we have a big problem, because
\begin{align}
y^\star =    \arg\max_y P(y|x, n)
\end{align}
is intractable as proven by [Sima'an (1996)](http://www.aclweb.org/anthology/C/C96/C96-2215.pdf).
Thus we typically approximate that criterion by
\begin{align}
y^\star =  \text{yield}_\Delta \left(  \arg\max_d P(y, d|x, n) \right)
\end{align}
which is often termed *Viterbi decoding*.

All this takes is a forest respresenting all translation candidates, i.e. \\(\mathcal D_n(x)\\) and a the Viterbi algorithm. Because in this project you need to implement *inside*, there's little point in programming Viterbi, you can simply use Inside for that. Check our [slides](https://uva-slpl.github.io/nlp2/resources/slides/crf.pdf) again.


